In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('laptop_price.csv', encoding="latin-1")
df = df.drop('Product', axis=1)
df = df.join(pd.get_dummies(df.Company))
df = df.drop('Company', axis=1)
df = df.join(pd.get_dummies(df.TypeName))
df = df.drop('TypeName', axis=1)
df['ScreenResolution'] = df.ScreenResolution.str.split(' ').apply(lambda x: x[-1])
df['Screen Width'] = df.ScreenResolution.str.split('x').apply(lambda x: x[0]) 
df['Screen Height'] = df.ScreenResolution.str.split('x').apply(lambda x: x[1]) 
df = df.drop('ScreenResolution', axis=1)
df['CPU Brand'] = df.Cpu.str.split(' ').apply(lambda x: x[0])
df['CPU Frequency'] = df.Cpu.str.split(' ').apply(lambda x: x[-1])
df = df.drop('Cpu', axis=1)
df['CPU Frequency'] = df['CPU Frequency'].str[:-3]
df['Ram'] = df['Ram'].str[:-2]
df['Ram'] = df['Ram'].astype('int')
df['CPU Frequency'] = df['CPU Frequency'].astype('float')
df['Screen Width'] = df['Screen Width'].astype('int')
df['Screen Height'] = df['Screen Height'].astype('int')
df['Memory Amount'] = df.Memory.str.split(' ').apply(lambda x: x[0])
df['Memory Type'] = df.Memory.str.split(' ').apply(lambda x: x[1])
def turn_memory_into_MB(value):
    if 'GB' in value:
        return float(value[:value.find('GB')]) * 1000
    elif 'TB' in value:
        return float(value[:value.find('TB')]) * 1000000
df['Memory Amount'] = df['Memory Amount'].apply(turn_memory_into_MB)
df = df.drop('Memory', axis=1)
df['Weight'] = df['Weight'].str[:-2]
df['Weight'] = df['Weight'].astype('float')
df['GPU Brand'] = df.Gpu.str.split(' ').apply(lambda x: x[0])
df = df.drop('Gpu', axis=1)
df = df.join(pd.get_dummies(df.OpSys))
df = df.drop('OpSys',axis=1)
cpu_catogories = pd.get_dummies(df['CPU Brand'])
cpu_catogories.columns = [col + '_CPU' for col in cpu_catogories.columns]

df = df.join(cpu_catogories)
df = df.drop('CPU Brand', axis=1)
gpu_catogories = pd.get_dummies(df['GPU Brand'])
gpu_catogories.columns = [col + '_GPU' for col in gpu_catogories.columns]

df = df.join(gpu_catogories)
df = df.drop('GPU Brand', axis=1)
target_correlations = df.corr()['Price_euros'].apply(abs).sort_values()
selected_feature = target_correlations[-21:].index
selected_feature = list(selected_feature)
limited_df = df[selected_feature]

#Train model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X, y = limited_df.drop('Price_euros', axis=1), limited_df['Price_euros']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
forest = RandomForestRegressor()

forest.fit(X_train_scaled, y_train)
forest.score(X_test_scaled, y_test)
y_pred = forest.predict(X_test_scaled)
X_test.iloc[:]

X_new_scaled = scaler.transform([X_test.iloc[0]])    

C:\Users\sb\AppData\Local\Temp\ipykernel_9300\2109303175.py:45: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  target_correlations = df.corr()['Price_euros'].apply(abs).sort_values()
c:\Users\sb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


prediction/estimated value

In [3]:
forest.predict(X_new_scaled)

array([956.243])

Actuall value

In [4]:
y_test.iloc[0]

1103.0